In [689]:
import os
from sshtunnel import SSHTunnelForwarder
import pg as db
import pandas as pd
import re
from sqlalchemy import create_engine
import numpy as np
import plotly as plotly
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline

host = 'prd-db.spot-internal'
localhost = '127.0.0.1'
ssh_username = os.environ['SPOT_SSH_USER']
ssh_private_key = '/Users/david/.ssh/id_rsa'
ssh_private_key_pwd = os.environ['SSH_PWD']

intermediary=os.environ['SPOT_SSH_SERVER']
user=os.environ['SPOT_DB_USER']
password=os.environ['SPOT_DB_PW']

database='trace_wood_delawareconnector'

def query(q):
    with SSHTunnelForwarder(
    (intermediary, 22), 
    ssh_username=ssh_username,
    ssh_pkey=ssh_private_key, 
    ssh_private_key_password=ssh_private_key_pwd,
    remote_bind_address=(host,5432)) as server:

        connstring = 'postgresql://{user}:{pwd}@{host}:{localport}/{database}'.format(
            user=user,
            pwd=password,
            host=localhost,
            localport=server.local_bind_port,
            database=database)
        
        engine = create_engine(connstring)

        try:
            result = pd.read_sql_query(q, engine)
        finally:
            engine.dispose()

        return result;

In [690]:
weld_numbers = query('select friendly_number as description from weld order by created_at asc')

In [723]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Create a simple method to quantify a weld string's characteristics. 

def weld_numerical_factor(description):
    alpha = hash(re.sub('[0-9]', '', description))
    numbers = re.sub('[^0-9]','',description);
    numeric = len(numbers)
    return [description, alpha, numeric]

features = pd.DataFrame(weld_numbers['description'].apply(weld_numerical_factor).array)

y = features[[0]]
X = features.drop(0, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

lof = LocalOutlierFactor(n_neighbors=3, novelty=True)
lof.fit(X_train)
y_pred = lof.predict(X_test)

y_test.reset_index()[pd.Series(y_pred) == -1]

,index,0
153,8781,TW-0005-NDE-WELDPASS
777,8773,TI-0404-COAT-NDE
805,8464,FAB-0002
